# Indexing Reward Unit Test
Performing verification of the indexing rewards mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

loaded GraphQL_data/SIstakeDelegateds.csv.
loaded GraphQL_data/SIstakeLockeds.csv.
loaded GraphQL_data/SIstakeWithdrawns.csv.
loaded GraphQL_data/SIallocationCollecteds.csv.
loaded GraphQL_data/SIstakeDepositeds.csv.


Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
stake_deposited_events_dict = stake_deposited_events[0]
print("EXPECTED TRUE INDEXING REWARDS FROM STAKE DEPOSITED EVENTS:")
is_first = True
for timestep, indexing_fee_events_list in stake_deposited_events_dict.items():
    if is_first:
        is_first = False
        pass
    else:        
        if indexing_fee_events_list is None:
            indexing_fee_amt = 0
        else:
            indexing_fee_amt = sum([e['tokens'] for e in indexing_fee_events_list])        
        print(indexing_fee_amt)
        # print(f"{timestep}, {stake_deposited_event[0]['tokens']}")
    


EXPECTED TRUE INDEXING REWARDS FROM STAKE DEPOSITED EVENTS:
6992.278838280839213400
7412.390565199642429400
0.314500799690706457
19917.334764642678146000
23772.682490475838525600
781.892419941371744000


In [4]:
print("MODELED RESULTS")
# print(df.iloc[timestep])
is_first = True
indexing_rewards_modeled = {}
for timestep, stake_deposited_event in stake_deposited_events_dict.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    if is_first:
        is_first = False
        pass
    else:       
        indexing_rewards =( df.iloc[timestep-1]['pool_delegated_stake'] - df.iloc[timestep-2]['pool_delegated_stake']) / (1 - indexer_revenue_cut[0]) * (indexer_revenue_cut[0])
        indexing_rewards_modeled[timestep] = indexing_rewards
        print(f"{timestep}, {indexing_rewards=}")


MODELED RESULTS
22, indexing_rewards=Decimal('6992.278838280839213400000035')
23, indexing_rewards=Decimal('7412.390565199642429399999997')
30, indexing_rewards=Decimal('0.3145007996907064569999597442')
59, indexing_rewards=Decimal('11791.92043340830915931947950')
128, indexing_rewards=Decimal('23772.68249047583852559999997')
130, indexing_rewards=Decimal('781.8924199413717440000000209')


In [10]:
print("UNITTEST RESULTS")
is_first = True
for timestep, stake_deposited_event in stake_deposited_events_dict.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    if is_first:
        is_first = False
        pass
    else:       
        indexing_rewards = indexing_rewards_modeled[timestep]
        print(f"Event: Timestep={timestep-1}, Indexing Reward Tokens={stake_deposited_event[0]['tokens']}")
        print(f"Model: Timestep={timestep-1}, Indexing Reward Tokens={indexing_rewards}")
        print("Tokens Equal?", stake_deposited_event[0]['tokens'] == indexing_rewards)
        print("Tokens Within 1%?", 0.99 <= abs(stake_deposited_event[0]['tokens'] / indexing_rewards) <= 1.01)
        print()


UNITTEST RESULTS
Event: Timestep=21, Indexing Reward Tokens=6992.278838280839213400
Model: Timestep=21, Indexing Reward Tokens=6992.278838280839213400000035
Shares Equal? False
Shares Within 1%? True

Event: Timestep=22, Indexing Reward Tokens=7412.390565199642429400
Model: Timestep=22, Indexing Reward Tokens=7412.390565199642429399999997
Shares Equal? False
Shares Within 1%? True

Event: Timestep=29, Indexing Reward Tokens=0.314500799690706457
Model: Timestep=29, Indexing Reward Tokens=0.3145007996907064569999597442
Shares Equal? False
Shares Within 1%? True

Event: Timestep=58, Indexing Reward Tokens=9556.847374872508219000
Model: Timestep=58, Indexing Reward Tokens=11791.92043340830915931947950
Shares Equal? False
Shares Within 1%? False

Event: Timestep=127, Indexing Reward Tokens=23772.682490475838525600
Model: Timestep=127, Indexing Reward Tokens=23772.68249047583852559999997
Shares Equal? False
Shares Within 1%? True

Event: Timestep=129, Indexing Reward Tokens=781.8924199413717

# Conclusion
The indexing reward token quantities are accurate to roughly 21 significant figures except in timestep 58.  